## 1. Libraries importing
Here we imported the pytorch library and required tools as we imported pandas for dataset manipulation.

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## This code is a simple application for a DNN implemented using Pytorch.
The pytorch network here is used for classification and here is the steps and how we define it :
# 1. Create a class for the network.

    class binaryClassification(nn.Module):
   -> Then : 
    
    def __init__(self):
    super(binaryClassification, self).__init__()

# 2. Define the layers needed in the netword. 
    
    self.layer_1 = nn.Linear(1, 64) 
    self.layer_2 = nn.Linear(64, 64)
    self.layer_out = nn.Linear(64, 1) 
        
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.1)
    self.batchnorm1 = nn.BatchNorm1d(64)
    self.batchnorm2 = nn.BatchNorm1d(64)

# 3. Define the 'forward' method in order to put the layers in their order.
        
    def forward(self, inputs):
           x = self.relu(self.layer_1(inputs))
           x = self.batchnorm1(x)
           x = self.relu(self.layer_2(x))
           x = self.batchnorm2(x)
           x = self.dropout(x)
           x = self.layer_out(x)
        
           return x

## 2. Import the dataset

In [2]:
data = pd.read_csv('/content/Social_Network_Ads.csv')
data = data[['Age','EstimatedSalary','Gender','Purchased']]
data.head()

,Age,EstimatedSalary,Gender,Purchased
0,19,19000,Male,0
1,35,20000,Male,0
2,26,43000,Female,0
3,27,57000,Female,0
4,19,76000,Male,0


In [3]:
data2 = pd.get_dummies(data['Gender'])
data['Gender'] = data2['Male']

In [4]:
data.head()

,Age,EstimatedSalary,Gender,Purchased
0,19,19000,1,0
1,35,20000,1,0
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,1,0


## 3. Creating the training and test sets

In [46]:
X = data[['Age','EstimatedSalary','Gender']]
y = data['Purchased']
X = X.values
y = y.values

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

In [48]:
from sklearn.preprocessing import StandardScaler    
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [49]:
X_test[:5]

array([[ 0.84621676, -1.45270451, -0.81907636],
       [ 0.01744058, -0.32835977, -0.81907636],
       [ 1.03038925,  1.89222109,  1.2208874 ],
       [-1.08759434,  0.37435569,  1.2208874 ],
       [ 0.938303  , -0.66566319, -0.81907636]])

In [50]:
from torch.utils.data import Dataset
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))

## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test))

In [51]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

## 4. Create the DNN

In [66]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(3, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [60]:
model = binaryClassification()
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)

binaryClassification(
  (layer_1): Linear(in_features=3, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [61]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [62]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 5. Train the DNN on the dataset

In [71]:
model.train()
EPOCHS = 50
model.eval()

for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X, y_batch in train_loader:
        X, y_batch = X.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    if (e%10 == 0):
      print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 010: | Loss: 0.13905 | Acc: 94.776
Epoch 020: | Loss: 0.12733 | Acc: 95.896
Epoch 030: | Loss: 0.12497 | Acc: 94.776
Epoch 040: | Loss: 0.11961 | Acc: 95.522
Epoch 050: | Loss: 0.11677 | Acc: 95.896


## 6. Test the DNN and print the final report

In [72]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [73]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.88      0.89      0.88        82
           1       0.82      0.80      0.81        50

    accuracy                           0.86       132
   macro avg       0.85      0.85      0.85       132
weighted avg       0.86      0.86      0.86       132

